In [145]:
from typing import Sequence

import clingo

In [146]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [147]:
commodities = """

commodity("Worker").
commodity("Material").

"""
solve([commodities]);

Answer 1: { commodity("Material") commodity("Worker") }
SAT 1


In [148]:
time = """

time(0..11).
startTime(ST) :- time(ST), not time(ST - 1).
endTime(ET) :- time(ET), not time(ET + 1).

"""
solve([time]);

Answer 1: { endTime(11) startTime(0) time(0) time(1) time(2) time(3) time(4) time(5) time(6) time(7) time(8) time(9) time(10) time(11) }
SAT 1


In [149]:
accounts = """

account_commodity_type("Hired:Workers", "Worker", logical).
account_commodity_type("Available:Workers", "Worker", real).
account_commodity_type("Busy:Workers", "Worker", logical).

account_commodity_type("Bought:Material", "Material", logical).
account_commodity_type("Available:Material", "Material", real).
account_commodity_type("Used:Materials", "Material", logical).

sourceAccount("Available:Material").
sourceAccount("Available:Workers").
sinkAccount("Used:Materials").
sinkAccount("Busy:Workers").

"""
solve([accounts]);

Answer 1: { sinkAccount("Busy:Workers") sinkAccount("Used:Materials") sourceAccount("Available:Material") sourceAccount("Available:Workers") account_commodity_type("Available:Material","Material",real) account_commodity_type("Available:Workers","Worker",real) account_commodity_type("Bought:Material","Material",logical) account_commodity_type("Busy:Workers","Worker",logical) account_commodity_type("Hired:Workers","Worker",logical) account_commodity_type("Used:Materials","Material",logical) }
SAT 1


In [150]:
init = """

obs_at(account_debits_credits_commodity(Account, 0, 0, Commodity), ST) :-
  startTime(ST),
  account_commodity_type(Account, Commodity, _Type).

"""
solve([init]);

Answer 1: {  }
SAT 1


<block>:4:3-16: info: atom does not occur in any rule head:
  startTime(ST)

<block>:5:3-52: info: atom does not occur in any rule head:
  account_commodity_type(Account,Commodity,_Type)



In [151]:
transaction = """

obs_at(account_debitChange_commodity(DebitAccount, Change, Commodity), T) :-
  time(T),
  account_commodity_type(DebitAccount, Commodity, _Type),
  Change > 0,
  Change = #sum{ Amount,CreditAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

obs_at(account_creditChange_commodity(CreditAccount, Change, Commodity), T) :-
  time(T),
  account_commodity_type(CreditAccount, Commodity, _Type),
  Change > 0,
  Change = #sum{ Amount,DebitAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

"""
solve([transaction]);

Answer 1: {  }
SAT 1


<block>:4:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-57: info: atom does not occur in any rule head:
  account_commodity_type(DebitAccount,Commodity,_Type)

<block>:7:41-143: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)

<block>:10:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:11:3-58: info: atom does not occur in any rule head:
  account_commodity_type(CreditAccount,Commodity,_Type)

<block>:13:40-142: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)



In [152]:
balance = """

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits + Credits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), T),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits + Credits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), T),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), T),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), T),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), T).

"""
solve([balance]);

Answer 1: {  }
SAT 1


<block>:4:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:5:3-52: info: atom does not occur in any rule head:
  account_commodity_type(Account,Commodity,_Type)

<block>:7:3-71: info: atom does not occur in any rule head:
  obs_at(account_debitChange_commodity(Account,Debits,Commodity),T)

<block>:8:3-73: info: atom does not occur in any rule head:
  obs_at(account_creditChange_commodity(Account,Credits,Commodity),T)

<block>:11:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:12:3-52: info: atom does not occur in any rule head:
  account_commodity_type(Account,Commodity,_Type)

<block>:14:7-70: info: atom does not occur in any rule head:
  obs_at(account_debitChange_commodity(#X0,#P1,#X2),#X3)

<block>:15:3-73: info: atom does not occur in any rule head:
  obs_at(account_creditChange_commodity(Account,Credits,Commodity),T)

<block>:18:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:19:3-52: info: atom does not oc

In [153]:
bim_rules = """

% Override Defaults
activity_duration(Activity, T) :- activity_setDuration(Activity, T).

activity_duration(construct(Wall), 2) :- product(wall, Wall), not activity_setDuration(construct(Wall), _).
activity_duration(connect(Wall1, Wall2), 1) :- product(wall, Wall1), product(wall, Wall2), not activity_setDuration(connect(Wall1, Wall2), _).

activity_requiresAmount_commodity(construct(Wall), 2, "Worker") :-
  product(wall, Wall), not activity_setRequiresAmount_commodity(construct(Wall), _, "Worker").
activity_consumesAmount_commodity(construct(Wall), 10, "Material") :-
  product(wall, Wall), not activity_setConsumesAmount_commodity(construct(Wall), _, "Material").

activity_requiresAmount_commodity(connect(Wall1, Wall2), 1, "Worker") :-
  product(wall, Wall1), product(wall, Wall2),
  not activity_setRequiresAmount_commodity(connect(Wall1, Wall2), _, "Worker").

activity_consumesAmount_commodity(connect(Wall1, Wall2), 1, "Material") :-
  product(wall, Wall1), product(wall, Wall2),
  not activity_setRequiresAmount_commodity(connect(Wall1, Wall2), _, "Material").

occ_at(debitAccount_creditAccount_amount_commodity(SinkAccount, SourceAccount, Amount, Commodity), T) :-
  occ_at(Activity, T),
  activity_requiresAmount_commodity(Activity, Amount, Commodity),
  sourceAccount(SourceAccount), account_commodity_type(SourceAccount, Commodity, _),
  sinkAccount(SinkAccount), account_commodity_type(SinkAccount, Commodity, _).

occ_at(debitAccount_creditAccount_amount_commodity(SourceAccount, SinkAccount, Amount, Commodity), T+D) :-
  occ_at(Activity, T), activity_duration(Activity, D),
  activity_requiresAmount_commodity(Activity, Amount, Commodity),
  sourceAccount(SourceAccount), account_commodity_type(SourceAccount, Commodity, _),
  sinkAccount(SinkAccount), account_commodity_type(SinkAccount, Commodity, _).

occ_at(debitAccount_creditAccount_amount_commodity(SinkAccount, SourceAccount, Amount, Commodity), T) :-
  occ_at(Activity, T),
  activity_consumesAmount_commodity(Activity, Amount, Commodity),
  sourceAccount(SourceAccount), account_commodity_type(SourceAccount, Commodity, _),
  sinkAccount(SinkAccount), account_commodity_type(SinkAccount, Commodity, _).


"""
solve([bim_rules]);

Answer 1: {  }
SAT 1


<block>:4:35-68: info: atom does not occur in any rule head:
  activity_setDuration(Activity,T)

<block>:6:42-61: info: atom does not occur in any rule head:
  product(wall,Wall)

<block>:6:67-107: info: atom does not occur in any rule head:
  activity_setDuration(construct(#X0),#P1)

<block>:7:48-68: info: atom does not occur in any rule head:
  product(wall,Wall1)

<block>:7:70-90: info: atom does not occur in any rule head:
  product(wall,Wall2)

<block>:7:96-142: info: atom does not occur in any rule head:
  activity_setDuration(connect(#X0,#X1),#P2)

<block>:10:3-22: info: atom does not occur in any rule head:
  product(wall,Wall)

<block>:10:28-94: info: atom does not occur in any rule head:
  activity_setRequiresAmount_commodity(construct(#X0),#P1,"Worker")

<block>:12:3-22: info: atom does not occur in any rule head:
  product(wall,Wall)

<block>:12:28-96: info: atom does not occur in any rule head:
  activity_setConsumesAmount_commodity(construct(#X0),#P1,"Material")

<block>:

In [154]:
instance = """

activity_setDuration(construct(wall2), 3).

product(wall, wall1).
product(wall, wall2).

occ_at(construct(wall1),1).
occ_at(construct(wall2),3).
occ_at(connect(wall1, wall2), 7).

occ_at(debitAccount_creditAccount_amount_commodity("Available:Workers", "Hired:Workers", 3, "Worker"), 0).
occ_at(debitAccount_creditAccount_amount_commodity("Available:Material", "Bought:Material", 21, "Material"), 0).

"""
solve([instance]);

Answer 1: { activity_setDuration(construct(wall2),3) occ_at(construct(wall1),1) occ_at(construct(wall2),3) occ_at(connect(wall1,wall2),7) occ_at(debitAccount_creditAccount_amount_commodity("Available:Material","Bought:Material",21,"Material"),0) occ_at(debitAccount_creditAccount_amount_commodity("Available:Workers","Hired:Workers",3,"Worker"),0) product(wall,wall1) product(wall,wall2) }
SAT 1


In [155]:
solve([
    commodities,
    time,
    accounts,
    init,
    transaction,
    balance,
    bim_rules,
    instance,
], sep='\n');

Answer 1: {
commodity("Material")
commodity("Worker")
endTime(11)
sinkAccount("Busy:Workers")
sinkAccount("Used:Materials")
sourceAccount("Available:Material")
sourceAccount("Available:Workers")
startTime(0)
time(0)
time(1)
time(2)
time(3)
time(4)
time(5)
time(6)
time(7)
time(8)
time(9)
time(10)
time(11)
activity_duration(construct(wall1),2)
activity_duration(construct(wall2),3)
activity_duration(connect(wall1,wall1),1)
activity_duration(connect(wall1,wall2),1)
activity_duration(connect(wall2,wall1),1)
activity_duration(connect(wall2,wall2),1)
activity_setDuration(construct(wall2),3)
obs_at(account_creditChange_commodity("Available:Material",1,"Material"),7)
obs_at(account_creditChange_commodity("Available:Material",10,"Material"),1)
obs_at(account_creditChange_commodity("Available:Material",10,"Material"),3)
obs_at(account_creditChange_commodity("Available:Workers",1,"Worker"),7)
obs_at(account_creditChange_commodity("Available:Workers",2,"Worker"),1)
obs_at(account_creditChange_commo

<block>:10:28-94: info: atom does not occur in any rule head:
  activity_setRequiresAmount_commodity(construct(#X0),#P1,"Worker")

<block>:12:28-96: info: atom does not occur in any rule head:
  activity_setConsumesAmount_commodity(construct(#X0),#P1,"Material")

<block>:16:7-79: info: atom does not occur in any rule head:
  activity_setRequiresAmount_commodity(connect(#X0,#X1),#P2,"Worker")

<block>:20:7-81: info: atom does not occur in any rule head:
  activity_setRequiresAmount_commodity(connect(#X0,#X1),#P2,"Material")

